This web-scraper of `https://data.covid19.go.id/public/index.html`
has update version at
- 2020/07/20 (first release)
- 2021/06/22

In [40]:
import time

import bs4
from bs4 import BeautifulSoup

from selenium import webdriver
import json

from pprint import pprint

# Get a specific type of news

download chromedriver version similar to your chrome browser   
https://sites.google.com/a/chromium.org/chromedriver/downloads

installation:
```
$ unzip chromedriver_linux64.zip
$ sudo mv chromedriver /usr/bin/chromedriver
$ sudo chown root:root /usr/bin/chromedriver
$ sudo chmod +x /usr/bin/chromedriver
```

In [6]:
base_site = "https://data.covid19.go.id/public/index.html"

options = webdriver.ChromeOptions()
options.add_argument("headless")

driver = webdriver.Chrome(executable_path="/usr/bin/chromedriver", 
                          options=options)
driver.get(base_site)

In [7]:
soup_page = BeautifulSoup(driver.page_source, parser='html5lib')
soup_page_str = str(soup_page)
#print(type(soup_page))
#print(soup_page.prettify())
soup_page_str

'<html lang="id"><head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1" name="viewport"/>\n<title>Situasi Virus Corona - Covid19.go.id</title>\n<link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" integrity="sha384-BVYiiSIFeK1dGmJRAkycuHAHRg32OmUcww7on3RYdg4Va+PmSTsz/K68vbdEjh4u" rel="stylesheet"/>\n<link href="css/main.css" rel="stylesheet"/>\n<link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet"/>\n<link href="https://cdn.jsdelivr.net/npm/select2@4.1.0-beta.1/dist/css/select2.min.css" rel="stylesheet"/>\n<link href="https://cdnjs.cloudflare.com/ajax/libs/select2-bootstrap-theme/0.1.0-beta.10/select2-bootstrap.min.css" rel="stylesheet"/>\n<link href="https://unpkg.com/leaflet@1.6.0/dist/leaflet.css" rel="stylesheet"/>\n<link href="https://cdnjs.cloudflare.com/ajax/libs/leaflet.markercluster/1.4.1/MarkerCluster.css" rel="stylesheet"/>\n<link hre

In [8]:
soup_page = BeautifulSoup(soup_page_str, parser='html5lib')

time_stamp = [data.get_text().split()[-1] 
              for data in soup_page.find_all('span', attrs={'class': 'pull-right'}) 
                  if 'Tanggal' in data.get_text()]
time_stamp

['2021-06-13']

In [9]:
print(f"Dataset is up to: {time_stamp[0]}")

Dataset is up to: 2021-06-13


In [77]:
headlines = [[headline_data.find('h3').string] + [data.string for data in headline_data.find_all('b')]
             for headline_data in soup_page.find_all('div', attrs={'class': 'col-md-3'})]

#pprint(headlines)

headlines_total_cases = {label.lower(): [total_case, increment.lower()]
                         for (total_case, label, increment) in [headlines[0]]}

temp = {}
for headline in headlines:
    if len(headline) == 4:
        total_case, _, label, increment = headline
        temp[label.lower()] = [total_case, increment.lower()]
    else:
        total_case, label, increment = headline
        temp[label.lower()] = [total_case, increment.lower()]
headlines = temp;

headlines

{'terkonfirmasi': ['2,018,113', '+13,668 kasus'],
 'kasus aktif': ['152,686', '+4,958 kasus aktif'],
 'sembuh': ['1,810,136', '+8,375 kasus sembuh'],
 'meninggal': ['55,291', '+335 kasus meninggal']}

In [78]:
for label, val in headlines.items():
    print(f"{label:>16s}: {val[0]} orang ({val[1]})")

   terkonfirmasi: 2,018,113 orang (+13,668 kasus)
     kasus aktif: 152,686 orang (+4,958 kasus aktif)
          sembuh: 1,810,136 orang (+8,375 kasus sembuh)
       meninggal: 55,291 orang (+335 kasus meninggal)


In [79]:
provinces_data = {data.find('b').get_text().strip().lower()\
                   : data.find('span').string[15:]
                  for data in soup_page.find_all('div', attrs={'class': 'wrapDetailInfo'})}
provinces_data

{'dki jakarta': '448,071 (23.4%)',
 'jawa barat': '328,940 (17.2%)',
 'jawa tengah': '215,684 (11.3%)',
 'jawa timur': '159,059 (8.3%)',
 'kalimantan timur': '72,701 (3.8%)',
 'riau': '65,740 (3.4%)',
 'sulawesi selatan': '62,672 (3.3%)',
 'banten': '51,201 (2.7%)',
 'daerah istimewa yogyakarta': '48,751 (2.6%)',
 'sumatera barat': '47,774 (2.5%)',
 'bali': '47,754 (2.5%)',
 'kalimantan selatan': '35,285 (1.8%)',
 'sumatera utara': '33,313 (1.7%)',
 'sumatera selatan': '26,223 (1.4%)',
 'kalimantan tengah': '23,763 (1.2%)',
 'papua': '20,601 (1.1%)',
 'kepulauan riau': '20,347 (1.1%)',
 'kepulauan bangka belitung': '19,701 (1.0%)',
 'lampung': '19,464 (1.0%)',
 'aceh': '17,376 (0.9%)',
 'nusa tenggara timur': '17,164 (0.9%)',
 'sulawesi utara': '15,860 (0.8%)',
 'sulawesi tengah': '13,080 (0.7%)',
 'nusa tenggara barat': '12,621 (0.7%)',
 'kalimantan utara': '12,468 (0.7%)',
 'kalimantan barat': '12,332 (0.6%)',
 'jambi': '11,015 (0.6%)',
 'sulawesi tenggara': '10,596 (0.6%)',
 'papua 

In [80]:
provinces_data['jawa timur']

'159,059 (8.3%)'

In [82]:
data = {'time_stamp': time_stamp, 
        'headlines': headlines,
        'provinces_data': provinces_data}

json_data = json.dumps(data, indent=4)

with open("covid_19.json", "w") as f:
    f.write(json_data)

In [83]:
with open("covid_19.json", "r") as f:
    data = f.read()
    
print(json.loads(data))

{'time_stamp': ['2021-06-13'], 'headlines': {'terkonfirmasi': ['2,018,113', '+13,668 kasus'], 'kasus aktif': ['152,686', '+4,958 kasus aktif'], 'sembuh': ['1,810,136', '+8,375 kasus sembuh'], 'meninggal': ['55,291', '+335 kasus meninggal']}, 'provinces_data': {'dki jakarta': '448,071 (23.4%)', 'jawa barat': '328,940 (17.2%)', 'jawa tengah': '215,684 (11.3%)', 'jawa timur': '159,059 (8.3%)', 'kalimantan timur': '72,701 (3.8%)', 'riau': '65,740 (3.4%)', 'sulawesi selatan': '62,672 (3.3%)', 'banten': '51,201 (2.7%)', 'daerah istimewa yogyakarta': '48,751 (2.6%)', 'sumatera barat': '47,774 (2.5%)', 'bali': '47,754 (2.5%)', 'kalimantan selatan': '35,285 (1.8%)', 'sumatera utara': '33,313 (1.7%)', 'sumatera selatan': '26,223 (1.4%)', 'kalimantan tengah': '23,763 (1.2%)', 'papua': '20,601 (1.1%)', 'kepulauan riau': '20,347 (1.1%)', 'kepulauan bangka belitung': '19,701 (1.0%)', 'lampung': '19,464 (1.0%)', 'aceh': '17,376 (0.9%)', 'nusa tenggara timur': '17,164 (0.9%)', 'sulawesi utara': '15,86

In [84]:
from threading import Thread
from multiprocessing import Process, Manager

class WorkerFunc(object):
    def __init__(self, queue_dict):
        self.queue_dict = queue_dict
        
    def worker(self, bs):
        a = [bs, "a", 1]
        self.queue_dict[0] = a
        time.sleep(3)
        
        
class CountdownTask: 
    def __init__(self): 
        self._running = True
      
    def terminate(self): 
        self._running = False
      
    def counter(self): 
        while self._running: 
            print(".", end="") 
            time.sleep(0.5) 

if __name__ == '__main__':
    manager = Manager()

    return_dict = manager.dict()
    print(return_dict.values())

    counter_fun = CountdownTask() 
    counter_proc = Thread(target=counter_fun.counter) 
    counter_proc.start() 
    #print(os.getpid())
    
    # Signal termination 
    worker_fun = WorkerFunc(return_dict)
    p1 = Process(target=worker_fun.worker,args=(bs4.BeautifulSoup, ))
    p1.start()
    p1.join()
    if not p1.is_alive():
        counter_fun.terminate()
        counter_proc.join()

    print(return_dict.values())

[]
.......[[<class 'bs4.BeautifulSoup'>, 'a', 1]]


In [85]:
import bs4
import sys
import time

from threading import Thread
from multiprocessing import Process, Manager

from bs4 import BeautifulSoup

from selenium import webdriver

#sys.setrecursionlimit(1000)

class WorkerTask(object):
    def __init__(self, queue_dict):
        self.queue_dict = queue_dict
        
    def worker(self):
        base_site = "https://data.covid19.go.id/public/index.html"
        
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        driver = webdriver.Chrome(executable_path="/usr/bin/chromedriver", options=options)
        driver.get(base_site)
        
        soup_page = BeautifulSoup(driver.page_source, parser="html5lib", features="lxml")
        
        self.queue_dict[0] = str(soup_page)
        #time.sleep(3)
        
        
class ProgressTask: 
    def __init__(self): 
        self._running = True
      
    def terminate(self): 
        self._running = False
      
    def print_point(self): 
        while self._running: 
            sys.stdout.write(".") 
            time.sleep(1) 


def scrap_covid19_go_id():
    queue_dict = Manager().dict()
    print(queue_dict.values())

    progress_ins = ProgressTask() 
    progress_proc = Thread(target=progress_ins.print_point) 
    progress_proc.start() 
    #print(os.getpid())
    
    # Signal termination 
    worker_ins = WorkerTask(queue_dict)
    worker_proc = Process(target=worker_ins.worker)
    worker_proc.start()
    worker_proc.join()
    
    if not worker_proc.is_alive():
        progress_ins.terminate()
        progress_proc.join()

    #print(queue_dict.values())
    soup_page = BeautifulSoup(queue_dict.values()[0], parser="lxml", features="lxml")
    
    return soup_page

            
if __name__ == '__main__':
    soup_page = scrap_covid19_go_id()
    #print(soup_page.prettify())
    #print(type(soup_page))

[]
.......

In [93]:
!python web-scraper.py -d True

Start scraping page of data.covid19.go.id/public/index.html
.......
Scrapping data.covid19.go.id finished.
Extraction finished.
-----------------------------
 Last updated at: 2021-06-13
-----------------------------
{'kasus aktif': ['152,686', '+4,958 Kasus Aktif'],
 'meninggal': ['55,291', '+335 Kasus Meninggal'],
 'sembuh': ['1,810,136', '+8,375 Kasus Sembuh'],
 'terkonfirmasi': ['2,018,113', '+13,668 Kasus']}
   terkonfirmasi: 2,018,113 orang (+13,668 Kasus)
     kasus aktif: 152,686 orang (+4,958 Kasus Aktif)
          sembuh: 1,810,136 orang (+8,375 Kasus Sembuh)
       meninggal:  55,291 orang (+335 Kasus Meninggal)
